In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql

In [2]:
#create connection to database file
database = "marketing-analysis-using-sql.db"
connection = sql.connect(database)

In [12]:
query = '''Select * FROM marketing_campaigns'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of    campaign_id          campaign_name  product_id  start_date    end_date
0            1            Summer Sale           2  2023-06-01  2023-06-30
1            2  New Collection Launch          10  2023-07-15  2023-08-15
2            3             Super Save           7  2023-08-20  2023-09-15>

In [9]:
query = '''Select * FROM sustainable_clothing'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of     product_id               product_name     category      size  price
0            1     Organic Cotton T-Shirt         Tops         S  29.99
1            2       Recycled Denim Jeans      Bottoms         M  79.99
2            3              Hemp Crop Top         Tops         L  24.99
3            4        Bamboo Lounge Pants      Bottoms        XS  49.99
4            5        Eco-Friendly Hoodie    Outerwear        XL  59.99
5            6    Linen Button-Down Shirt         Tops         M  39.99
6            7       Organic Cotton Dress      Dresses         S  69.99
7            8    Sustainable Swim Shorts     Swimwear         L  34.99
8            9  Recycled Polyester Jacket    Outerwear        XL  89.99
9           10       Bamboo Yoga Leggings   Activewear        XS  54.99
10          11              Hemp Overalls      Bottoms         M  74.99
11          12     Organic Cotton Sweater         Tops         L  49.99
12          13               Cork 

In [11]:
query = '''Select * FROM transactions'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of     transaction_id  product_id  quantity purchase_date
0                1           2         2    2023-06-02
1                2          14         1    2023-06-02
2                3           5         2    2023-06-05
3                4           2         1    2023-06-07
4                5          19         2    2023-06-10
..             ...         ...       ...           ...
59              60          13         1    2023-10-13
60              61          10         2    2023-10-13
61              62           9         1    2023-10-13
62              63          19         2    2023-10-13
63              64          20         1    2023-10-14

[64 rows x 4 columns]>

In [13]:
# 1. How many transactions were completed during each marketing campaign

query='''SELECT campaign_name,count(transaction_id) as Transactions
FROM transactions
JOIN marketing_campaigns
ON purchase_date
WHERE purchase_date BETWEEN start_date and end_date
GROUP BY campaign_name;'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of            campaign_name  Transactions
0  New Collection Launch             9
1            Summer Sale            13
2             Super Save             8>

In [14]:
# 2. Which product had the highest sales quantity?

query='''SELECT product_name , sum(quantity) as Sales_quantity
FROM sustainable_clothing
JOIN transactions
USING (product_id)
GROUP BY product_name
ORDER BY Sales_quantity DESC
LIMIT 1;'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of              product_name  Sales_quantity
0  Organic Cotton Sweater               9>

In [15]:
# 3. What is the total revenue generated from each marketing campaign?

query='''SELECT campaign_name,round(sum(quantity * price),2) as Total_Revenue
FROM transactions as t
JOIN marketing_campaigns 
ON purchase_date
BETWEEN start_date and end_date
JOIN sustainable_clothing as s 
ON s.product_id = t.product_id
GROUP BY campaign_name;'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of            campaign_name  Total_Revenue
0  New Collection Launch         499.89
1            Summer Sale        1044.82
2             Super Save         529.89>

In [16]:
# 4. What is the top-selling product category based on the total revenue generated?

query='''SELECT category,round(sum(price *quantity),2) as Total_revenue
FROM transactions as t
JOIN sustainable_clothing as s 
ON s.product_id = t.product_id
GROUP BY category
ORDER BY Total_revenue DESC
LIMIT 1;'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of   category  Total_revenue
0  Bottoms        1289.79>

In [17]:
# 5. Which products had a higher quantity sold compared to the average quantity sold?

query='''SELECT s.product_id,product_name FROM sustainable_clothing as s
WHERE s.product_id IN (
SELECT product_id FROM 
(SELECT product_id,sum(quantity) as quant
FROM transactions as t
GROUP BY product_id) as t1
WHERE quant 
>(
SELECT AVG(quant)
FROM
(SELECT sum(quantity) as quant
FROM transactions as t
GROUP BY product_id)as t2
) 
);'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of    product_id            product_name
0           2    Recycled Denim Jeans
1           4     Bamboo Lounge Pants
2           5     Eco-Friendly Hoodie
3          10    Bamboo Yoga Leggings
4          12  Organic Cotton Sweater
5          15    Organic Cotton Skirt
6          16       Hemp Baseball Cap
7          18          Linen Jumpsuit
8          19    Organic Cotton Socks>

In [18]:
# 6. What is the average revenue generated per day during the marketing campaigns?

query='''SELECT purchase_date,round(AVG(quantity * price),2) as Average_Revenue
FROM transactions as t
JOIN marketing_campaigns 
ON purchase_date
BETWEEN start_date and end_date
JOIN sustainable_clothing as s 
ON s.product_id = t.product_id
GROUP BY purchase_date;'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of    purchase_date  Average_Revenue
0     2023-06-02           109.99
1     2023-06-05           119.98
2     2023-06-07            79.99
3     2023-06-10            19.98
4     2023-06-13            52.49
5     2023-06-15           109.98
6     2023-06-18            79.99
7     2023-06-22            49.99
8     2023-06-26           139.98
9     2023-06-30            59.99
10    2023-07-16            74.99
11    2023-07-20            54.99
12    2023-07-24            99.98
13    2023-07-29            57.49
14    2023-08-03            54.99
15    2023-08-08            19.98
16    2023-08-14            39.99
17    2023-08-20            49.98
18    2023-08-27            69.99
19    2023-09-01            99.98
20    2023-09-05            54.99
21    2023-09-10            39.99
22    2023-09-14            79.99>

In [19]:
# 7. What is the percentage contribution of each product to the total revenue?

query='''with table2 as(
With table1 as(
SELECT product_name,round(sum(quantity * price),2) as Product_Revenue
FROM transactions as t
JOIN sustainable_clothing as s 
ON s.product_id = t.product_id
GROUP BY product_name)
SELECT *,sum(Product_Revenue) OVER() as Total_Revenue
    FROM table1)
SELECT product_name, round((Product_Revenue/Total_Revenue) * 100 ,2) as percentage_contribution 
FROM table2
ORDER BY percentage_contribution DESC;'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of                  product_name  percentage_contribution
0        Recycled Denim Jeans                    13.71
1              Linen Jumpsuit                    10.49
2      Organic Cotton Sweater                     9.64
3        Bamboo Yoga Leggings                     9.42
4   Recycled Polyester Jacket                     7.71
5         Eco-Friendly Hoodie                     6.42
6         Bamboo Lounge Pants                     5.35
7       Upcycled Denim Jacket                     5.14
8               Hemp Overalls                     4.82
9        Organic Cotton Dress                     4.50
10       Organic Cotton Skirt                     3.75
11               Cork Sandals                     3.43
12            Bamboo Bathrobe                     3.00
13          Hemp Baseball Cap                     2.68
14    Linen Button-Down Shirt                     2.57
15    Recycled Nylon Backpack                     2.57
16       Organic Cotton Socks      

In [20]:
# 8. Compare the average quantity sold during marketing campaigns to outside the marketing campaigns

query='''With table2 as (SELECT AVG(quantity) as Total_sales 
                FROM transactions),
table1 as(
SELECT AVG(quantity) as AVG_Sales_Marketing_campaigns
FROM transactions as t
JOIN sustainable_clothing as s
ON t.product_id = s.product_id
JOIN marketing_campaigns as m
ON purchase_date 
BETWEEN start_date and end_date)
SELECT *,Total_sales - AVG_Sales_Marketing_campaigns As outside_Marketing_campaigns FROM table2,table1;'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of    Total_sales  AVG_Sales_Marketing_campaigns  outside_Marketing_campaigns
0        1.375                       1.333333                     0.041667>

In [21]:
# 9. Compare the revenue generated by products inside the marketing campaigns to outside the campaigns

query='''With table2 as (SELECT round(SUM(price *quantity),2) as Total_sales 
                FROM transactions as t2
                JOIN sustainable_clothing as s
                ON t2.product_id = s.product_id),
table1 as(
SELECT round(SUM(price *quantity ),2) as Product_Sales_Marketing_campaigns
FROM transactions as t3
JOIN sustainable_clothing as s
ON t3.product_id = s.product_id
JOIN marketing_campaigns as m
ON purchase_date 
BETWEEN start_date and end_date)
SELECT Total_sales,Product_Sales_Marketing_campaigns,Total_sales -Product_Sales_Marketing_campaigns
FROM table2,table1;'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of    Total_sales  Product_Sales_Marketing_campaigns  \
0      4669.12                             2074.6   

   Total_sales -Product_Sales_Marketing_campaigns  
0                                         2594.52  >

In [22]:
# 10. Rank the products by their average daily quantity sold
 
query='''with table2 as(
With table1 as(
SELECT product_name,product_id,sum(quantity) OVER(partition by purchase_date) as daily_qty
FROM sustainable_clothing
JOIN transactions
USING(product_id))
SELECT DISTINCT product_name,ROUND(AVG(daily_qty) OVER(partition by product_name)) as AVG_daily 
    FROM table1)
SELECT *,dense_rank() OVER(order by AVG_daily DESC) as Ranking FROM table2;'''

df = pd.read_sql_query(query, connection)
df.head

<bound method NDFrame.head of                  product_name  AVG_daily  Ranking
0   Recycled Polyester Jacket        5.0        1
1       Upcycled Denim Jacket        5.0        1
2        Organic Cotton Socks        4.0        2
3      Organic Cotton T-Shirt        4.0        2
4     Sustainable Swim Shorts        4.0        2
5        Bamboo Yoga Leggings        3.0        3
6                Cork Sandals        3.0        3
7           Hemp Baseball Cap        3.0        3
8              Linen Jumpsuit        3.0        3
9        Organic Cotton Skirt        3.0        3
10     Organic Cotton Sweater        3.0        3
11    Recycled Nylon Backpack        3.0        3
12            Bamboo Bathrobe        2.0        4
13        Bamboo Lounge Pants        2.0        4
14        Eco-Friendly Hoodie        2.0        4
15              Hemp Crop Top        2.0        4
16              Hemp Overalls        2.0        4
17    Linen Button-Down Shirt        2.0        4
18       Recycled De